# Saving and loading populations

Another feature Brush implements is the ability to save and load entire populations.
We use JSON notation to store the population into a file that is human readable. The same way, we can feed an estimator a previous population file to serve as starting point for the evolution.

In this notebook, we will walk through how to use the `save_population` and `load_population` parameters. 

We start by getting a sample dataset and splitting it into `X` and `y`:

In [25]:
import pandas as pd
from pybrush import BrushRegressor

# load data
df = pd.read_csv('../examples/datasets/d_enc.csv')
X = df.drop(columns='label')
y = df['label']

To save the population after finishing the evolution, you nee to set `save_population` parameter to a value different than an empty string. Then, the final population is going to be stored in that specific file.

In this example, we create a temporary file.

In [28]:
import pickle
import os, tempfile

pop_file = os.path.join(tempfile.mkdtemp(), 'population.json')

# set verbosity==2 to see the full report
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    max_gens=10,
    objectives=["error", "complexity"],
    save_population=pop_file,
    use_arch=True, # Only the pareto front of last gen will be stored in archive
    verbosity=2
)

est.fit(X,y)
y_pred = est.predict(X)
print('score:', est.score(X,y))

Generation 1/10 [//////                                            ]
Train Loss (Med): 14.45158 (74.37033)
Val Loss (Med): 14.45158 (74.37033)
Median Size (Max): 3 (24)
Median complexity (Max): 9 (3315)
Time (s): 0.13028

Generation 2/10 [///////////                                       ]
Train Loss (Med): 14.15208 (20.54475)
Val Loss (Med): 14.15208 (20.54475)
Median Size (Max): 3 (19)
Median complexity (Max): 9 (1671)
Time (s): 0.20999

Generation 3/10 [////////////////                                  ]
Train Loss (Med): 11.74184 (17.94969)
Val Loss (Med): 11.74184 (17.94969)
Median Size (Max): 3 (19)
Median complexity (Max): 9 (1671)
Time (s): 0.30145

Generation 4/10 [/////////////////////                             ]
Train Loss (Med): 11.33994 (17.94969)
Val Loss (Med): 11.33994 (17.94969)
Median Size (Max): 3 (19)
Median complexity (Max): 9 (975)
Time (s): 0.41635

Generation 5/10 [//////////////////////////                        ]
Train Loss (Med): 10.79139 (90.38514)
Val Lo

Loading a previous population is done providing `load_population` a string value corresponding to a JSON file generated by Brush. In our case, we will use the same file from the previous code block.

After loading the population, we run the evolution for 10 more generations, and we can see that the first generation started from the previous population. This means that the population was successfully saved and loaded.

In [3]:
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    load_population=pop_file,
    max_gens=10,
    verbosity=1
)

est.fit(X,y)
y_pred = est.predict(X)
print('score:', est.score(X,y))

Loaded population from /tmp/tmphau_4iem/population.json of size = 200
Completed 100% [====================]
saving final population as archive...
score: 0.8864496413943844


## Saving just the archive

In case you want to use another expression rather than the final `best_estimator_`, brush provides the archive option.

The archive is just the pareto front from the population. You can use `predict_archive` (and `predict_proba_archive` if using a `BrushClassifier`) to call the prediction methods for the entire archive, instead of the selected best individual.

But first, you need to enable this option with `use_arch=True`. When set to False, it will store the entire final population

In [4]:
est = BrushRegressor(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    load_population=pop_file,
    use_arch=True,
    max_gens=10,
    verbosity=1
)

est.fit(X,y)

# accessing first expression from the archive. It is serialized as a dict
print(est.archive_[0]['fitness'])

Loaded population from /tmp/tmphau_4iem/population.json of size = 200
Completed 100% [====================]
{'complexity': 164, 'crowding_dist': 3.4028234663852886e+38, 'dcounter': 0, 'depth': 3, 'dominated': [], 'linear_complexity': 16, 'loss': 10.66516399383545, 'loss_v': 10.66516399383545, 'rank': 1, 'size': 13, 'values': [10.66516399383545, 13.0], 'weights': [-1.0, -1.0], 'wvalues': [-10.66516399383545, -13.0]}


You can open the serialized file and change individuals' programs manually.

This also allow us to have checkpoints in the execution.

## Using population files with classification

To give another example, we do a two-step fit in the cells below.

First, we run the evolution and save the population to a file; then, we load it and keep evolving the individuals.

What is different though is that the first run is optimizing `error` and `complexity`, and the second run is optimizing `average_precision_score` and `linear_complexity`.

In [31]:
from pybrush import BrushClassifier

# load data
df = pd.read_csv('../examples/datasets/d_analcatdata_aids.csv')
X = df.drop(columns='target')
y = df['target']

pop_file = os.path.join(tempfile.mkdtemp(), 'population.json')

est = BrushClassifier(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    max_gens=10,
    objectives=["error", "complexity"],
    scorer="log",
    save_population=pop_file,
    pop_size=1000,
    verbosity=2
)

est.fit(X,y)

print("Best model:", est.best_estimator_.get_model())
print('score:', est.score(X,y))

Generation 1/10 [//////                                            ]
Train Loss (Med): 0.62865 (0.69315)
Val Loss (Med): 0.62865 (0.69315)
Median Size (Max): 7 (12)
Median complexity (Max): 21 (5889)
Time (s): 1.23444

Generation 2/10 [///////////                                       ]
Train Loss (Med): 0.62321 (0.69315)
Val Loss (Med): 0.62321 (0.69315)
Median Size (Max): 7 (12)
Median complexity (Max): 21 (5889)
Time (s): 2.44751

Generation 3/10 [////////////////                                  ]
Train Loss (Med): 0.62114 (0.69315)
Val Loss (Med): 0.62114 (0.69315)
Median Size (Max): 7 (12)
Median complexity (Max): 21 (5889)
Time (s): 3.66024

Generation 4/10 [/////////////////////                             ]
Train Loss (Med): 0.62022 (0.69315)
Val Loss (Med): 0.62022 (0.69315)
Median Size (Max): 7 (12)
Median complexity (Max): 21 (5889)
Time (s): 4.99399

Generation 5/10 [//////////////////////////                        ]
Train Loss (Med): 0.61990 (0.69315)
Val Loss (Med): 0.6

In [32]:
from sklearn.metrics import accuracy_score

accuracy_score(y, est.predict(X))

0.6

In [33]:
est = BrushClassifier(
    functions=['SplitBest','Add','Mul','Sin','Cos','Exp','Logabs'],
    load_population=pop_file,
    objectives=["error", "complexity"],
    scorer="average_precision_score",
    max_gens=10,
    validation_size=0.0,
    pop_size=1000, # make sure this is the same as loaded pop
    use_arch=True,
    verbosity=1
)

est.fit(X,y)

print("Best model:", est.best_estimator_.get_model())
print('score:', est.score(X,y))

Loaded population from /tmp/tmp6z9ftnp3/population.json of size = 2000
Completed 100% [====================]
Best model: 1.00*Logistic(Sum(0.49994996,If(AIDS>15890.50,1.00*MeanLabel,0.00*MeanLabel)))
score: 0.5


We can see the fitness object, and that the error now matches the average precision score metric:

In [34]:
# Fitness is (error, linear complexity)
print(est.best_estimator_.fitness)

Fitness(0.680010 489.000000 )


In [35]:
from sklearn.metrics import average_precision_score

# takes y_true as first argument, and y_pred as second argument.
average_precision_score(y, est.predict_proba(X)[:, 1]) #, average='weighted')

0.6799999999999999